# Step one: Data analysis and cleaning

In [11]:
import pandas as pd
import numpy as np
import ast

In [12]:
# Reading the main data
df = pd.read_csv('games.csv')

print(f'Shape: {df.shape}')
df.columns

Shape: (51490, 61)


Index(['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner',
       'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron',
       'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1',
       't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2',
       't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id',
       't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1',
       't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3',
       't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2',
       't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id',
       't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1',
       't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHer

In [13]:
# Reading the champions json
import json
f = open('champion_info.json')
champs = json.load(f)
rows = []
champs_df = pd.DataFrame(columns=['id', 'name'])
for champion_id, champion_data in champs['data'].items():
    champ_id = int(champion_id)
    name = champion_data['name']
    rows.append({'id': champ_id, 'name': name})
    
champs_df['name'] = [row['name'] for row in rows]
champs_df['id'] = [row['id'] for row in rows]
champs_df
    



,id,name
0,1,Annie
1,2,Olaf
2,3,Galio
3,4,Twisted Fate
4,5,Xin Zhao
...,...,...
133,429,Kalista
134,432,Bard
135,497,Rakan
136,498,Xayah


In [14]:
# Note that there arent any null values since the data was previously cleaned by its author
df.isna().sum().unique()

array([0], dtype=int64)

Analyzing champions' win percentage and pick/ban rate

In [15]:
# Analyzing champions' win percentage and pick/ban rate

champs_t1, champs_t2, bans_t1, bans_t2 = [],[],[],[]

# Getting columns that contains champions' appearances 'automatically'

# When testing, checking whether its alread in champ_t1 became necessary since compyling it twice
# would duplicate the list's values
for column in df.columns:
    if 'id' in column:
        if 't1' in column and column not in champs_t1:
            champs_t1.append(column)
        elif 't2' in column and column not in champs_t2:
            champs_t2.append(column)
    if 'ban' in column:
        if 't1' in column and column not in bans_t1:
            bans_t1.append(column)
        elif 't2' in column and column not in bans_t2:
            bans_t2.append(column)

In [17]:
df.shape[0]

51490

In [25]:
# Extracting picked/banned percentage and getting the relevance:
# how many times a champion is picked when its not banned
# (number of picks) / ((total games) - (number of bans))
# In my opinion, this number would infer the best bans

df['picks'] = df[champs_t1 + champs_t2].values.tolist()
df['bans'] = df[bans_t1 + bans_t2].values.tolist()

# I struggled a lot to assign lists/arrays to specific cells, so here's the solution i came up
df['win/lose'] = np.where(df['winner'] == 1, '[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]', '[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]')
df['win/lose'] = df['win/lose'].apply(ast.literal_eval)
df_exploded = df.explode(['picks','bans','win/lose'])

pick_id = df_exploded['picks'].value_counts().reset_index()
ban_id = df_exploded['bans'].value_counts().reset_index()
winrate_id = df_exploded[['picks','win/lose']].groupby('picks').sum().reset_index()
winrate_id.rename(columns={'picks': 'index'},inplace=True)


# Since pd.merge only works for 2 dfs, had to do a step inside it
df_champion_stats = pd.merge(pick_id,pd.merge(ban_id,winrate_id,on='index'),on='index')
df_champion_stats['picked or banned (%)'] = (df_champion_stats['picks'] + df_champion_stats['bans'])*100/df.shape[0]
df_champion_stats['relevance (%)'] = df_champion_stats['picks']/(df.shape[0]-df_champion_stats['bans'])*100

df_exploded['picks'].value_counts().reset_index()

df_champion_stats.rename(columns={'index': 'id'}, inplace= True)
df_champion_stats = pd.merge(champs_df,df_champion_stats,on='id')

df_champion_stats[['picked or banned (%)', 'relevance (%)']] = df_champion_stats[['picked or banned (%)', 'relevance (%)']].round(2)
df_champion_stats.sort_values(by='relevance (%)', ascending = False)
df_champion_stats['winrate (%)'] = round(100*df_champion_stats['win/lose']/df_champion_stats['picks'],2)

df_champion_stats = df_champion_stats.sort_values('winrate (%)', ascending=False)

df_champion_stats

,id,name,picks,bans,win/lose,picked or banned (%),relevance (%),winrate (%)
39,40,Janna,8691,21390,4826,58.42,28.87,55.53
36,37,Sona,5429,613,2942,11.73,10.67,54.19
73,83,Yorick,1378,505,744,3.66,2.70,53.99
32,33,Rammus,2997,1846,1614,9.41,6.04,53.85
33,34,Anivia,2252,877,1207,6.08,4.45,53.60
...,...,...,...,...,...,...,...,...
68,78,Poppy,1886,161,852,3.98,3.67,45.17
42,43,Karma,3134,215,1415,6.50,6.11,45.15
128,268,Azir,1280,419,555,3.30,2.51,43.36
137,516,Ornn,4766,5685,1955,20.30,10.40,41.02
